In [191]:
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain.chains import LLMChain
from langchain.prompts import ChatPromptTemplate
from langchain_core.output_parsers import JsonOutputParser, CommaSeparatedListOutputParser, StrOutputParser
import boto3
import os
import re
import json
from datetime import datetime
from pydantic import BaseModel
import importlib
import shutil
from utils import utils
import pandas as pd
from utils.time_converter import TimeConverter
importlib.reload(utils)
load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
DEEPSEEK_API_KEY = os.getenv("DEEPSEEK_API_KEY")

In [192]:
subject = '수학'
room_id = '674e58163925f28f6caf4fa0'

file_keys = utils.get_items('pagecall-text', f'{subject}/{room_id}')
utils.download_items('pagecall-text', file_keys, './downloads')

In [193]:
raw_data = utils.merge_files('./downloads')
shutil.rmtree('./downloads')

In [194]:
teacher_extracted_data = utils.extract_speaker(raw_data, speaker='teacher')
teacher_splited_data = utils.split_sentences(teacher_extracted_data)
student_extracted_data = utils.extract_speaker(raw_data, speaker='student')
student_splited_data = utils.split_sentences(student_extracted_data)

In [195]:
teacher_splited_data = utils.mapping_time(teacher_extracted_data, teacher_splited_data)
teacher_splited_data

[{'idx': 0,
  'text': '목소리 잘 들릴까요?',
  'time': '4m 54.5s ~ 4m 58.3s',
  'start': '2024-12-09 10:54:41.68',
  'end': '2024-12-09 10:54:45.48'},
 {'idx': 1,
  'text': '안녕하세요.',
  'time': '4m 58.5s ~ 5m 0.6s',
  'start': '2024-12-09 10:54:45.68',
  'end': '2024-12-09 10:54:47.78'},
 {'idx': 2,
  'text': '잘 들리고 그러면 필기 한번 확인해 볼게요.',
  'time': '5m 0.7s ~ 5m 6.2s',
  'start': '2024-12-09 10:54:47.88',
  'end': '2024-12-09 10:54:53.38'},
 {'idx': 3,
  'text': '잘 보여요?',
  'time': '5m 9.7s ~ 5m 12.5s',
  'start': '2024-12-09 10:54:56.88',
  'end': '2024-12-09 10:54:59.68'},
 {'idx': 4,
  'text': '잠깐만요.',
  'time': '5m 12.5s ~ 5m 12.6s',
  'start': '2024-12-09 10:54:59.68',
  'end': '2024-12-09 10:54:59.78'},
 {'idx': 5,
  'text': '혹시 전에 선택으로 과외 해본 적 있어요?',
  'time': '5m 23.3s ~ 5m 35.5s',
  'start': '2024-12-09 10:55:10.48',
  'end': '2024-12-09 10:55:22.68'},
 {'idx': 6,
  'text': '아 정말요?',
  'time': '5m 35.5s ~ 5m 38.1s',
  'start': '2024-12-09 10:55:22.68',
  'end': '2024-12-09 10:55:25.28'},

In [196]:
student_splited_data = utils.mapping_time(student_extracted_data, student_splited_data)
student_splited_data

[{'idx': 0,
  'text': '감사합니다.',
  'time': '0m 0.0s ~ 0m 2.4s',
  'start': '2024-12-09 10:49:47.15',
  'end': '2024-12-09 10:49:49.55'},
 {'idx': 1,
  'text': '네 안녕하세요',
  'time': '4m 41.8s ~ 5m 10.4s',
  'start': '2024-12-09 10:54:28.99',
  'end': '2024-12-09 10:54:57.59'},
 {'idx': 2,
  'text': '네 네',
  'time': '4m 41.8s ~ 5m 10.4s',
  'start': '2024-12-09 10:54:28.99',
  'end': '2024-12-09 10:54:57.59'},
 {'idx': 3,
  'text': '네, 같이 볼게요.',
  'time': '5m 35.0s ~ 5m 55.5s',
  'start': '2024-12-09 10:55:22.19',
  'end': '2024-12-09 10:55:42.69'},
 {'idx': 4,
  'text': '어떤 걸 보여줄지 모르겠어가지고..',
  'time': '6m 4.0s ~ 6m 33.1s',
  'start': '2024-12-09 10:55:51.19',
  'end': '2024-12-09 10:56:20.29'},
 {'idx': 5,
  'text': '아니 근데..',
  'time': '6m 4.0s ~ 6m 33.1s',
  'start': '2024-12-09 10:55:51.19',
  'end': '2024-12-09 10:56:20.29'},
 {'idx': 6,
  'text': '근데 솔직히 볼 수 있는 게 몇 개 없어가지고..',
  'time': '6m 4.0s ~ 6m 33.1s',
  'start': '2024-12-09 10:55:51.19',
  'end': '2024-12-09 10:56:20.29'},
 {

In [197]:
teacher_df = pd.DataFrame(teacher_splited_data).rename(columns={"idx": "teacher_idx", "text": "teacher_text"})
student_df = pd.DataFrame(student_splited_data).rename(columns={"idx": "student_idx", "text": "student_text"})

df = pd.concat([teacher_df, student_df], ignore_index=True)
df = df.sort_values(by=["start", "teacher_idx", "student_idx"]).reset_index(drop=True)
df = df.astype({'teacher_idx': 'Int64', 'student_idx': 'Int64'})
df = df[['start', 'end', 'teacher_idx', 'student_idx', 'time', 'teacher_text', 'student_text']]

teacher_df = df[df['teacher_text'].notnull()].drop(columns=['student_text', 'student_idx', 'start', 'end', 'time']).rename(columns={"teacher_idx": "idx", "teacher_text": "text"}).reset_index(drop=True)
student_df = df[df['student_text'].notnull()].drop(columns=['teacher_text', 'teacher_idx', 'start', 'end', 'time']).rename(columns={"student_idx": "idx", "student_text": "text"}).reset_index(drop=True)
df

,start,end,teacher_idx,student_idx,time,teacher_text,student_text
0,2024-12-09 10:49:47.15,2024-12-09 10:49:49.55,<NA>,0,0m 0.0s ~ 0m 2.4s,NaN,감사합니다.
1,2024-12-09 10:54:28.99,2024-12-09 10:54:57.59,<NA>,1,4m 41.8s ~ 5m 10.4s,NaN,네 안녕하세요
2,2024-12-09 10:54:28.99,2024-12-09 10:54:57.59,<NA>,2,4m 41.8s ~ 5m 10.4s,NaN,네 네
3,2024-12-09 10:54:41.68,2024-12-09 10:54:45.48,0,<NA>,4m 54.5s ~ 4m 58.3s,목소리 잘 들릴까요?,NaN
4,2024-12-09 10:54:45.68,2024-12-09 10:54:47.78,1,<NA>,4m 58.5s ~ 5m 0.6s,안녕하세요.,NaN
...,...,...,...,...,...,...,...
1377,2024-12-09 12:47:39.47,2024-12-09 12:47:39.77,<NA>,440,117m 52.3s ~ 117m 52.6s,NaN,안녕히 계세요.
1378,2024-12-09 12:47:54.88,2024-12-09 12:47:55.58,937,<NA>,118m 7.7s ~ 118m 8.4s,안녕하세요.,NaN
1379,2024-12-09 12:47:55.68,2024-12-09 12:47:58.18,938,<NA>,118m 8.5s ~ 118m 11.0s,처음인데 미안해요.,NaN
1380,2024-12-09 12:47:58.28,2024-12-09 12:47:59.08,939,<NA>,118m 11.1s ~ 118m 11.9s,미안하네요.,NaN


In [198]:
teacher_df

,idx,text
0,0,목소리 잘 들릴까요?
1,1,안녕하세요.
2,2,잘 들리고 그러면 필기 한번 확인해 볼게요.
3,3,잘 보여요?
4,4,잠깐만요.
...,...,...
936,936,네 그냥 나가면 되는 걸까요?
937,937,안녕하세요.
938,938,처음인데 미안해요.
939,939,미안하네요.


In [199]:
chunks_with_overlap = utils.split_with_overlap(teacher_df, chunk_size=30, overlap=5)
chunks_with_overlap

[[{'idx': 0, 'text': '목소리 잘 들릴까요?'},
  {'idx': 1, 'text': '안녕하세요.'},
  {'idx': 2, 'text': '잘 들리고 그러면 필기 한번 확인해 볼게요.'},
  {'idx': 3, 'text': '잘 보여요?'},
  {'idx': 4, 'text': '잠깐만요.'},
  {'idx': 5, 'text': '혹시 전에 선택으로 과외 해본 적 있어요?'},
  {'idx': 6, 'text': '아 정말요?'},
  {'idx': 7, 'text': '저 되게 첫 수업이라서 조금 떨리는데 이거 화면 바뀔 때 같이 바뀌나요?'},
  {'idx': 8, 'text': '이거 레벨테스트 봤는데 이거 두 개를 풀어줬더라구요'},
  {'idx': 9, 'text': '아 그거 괜찮아요'},
  {'idx': 10,
   'text': '같이 하면 되니까 이게 왜 두 개였냐면 이게 상반기용이 하나 있었고, 하반기용이 하나 있었거든요.'},
  {'idx': 11, 'text': '그래서 이 파트가 지금 하반기용이고, 이 도형이 많은 게 상반기용이더라고요.'},
  {'idx': 12,
   'text': '그래서 일단은 지금 하반기니까, 하반기 거를 일단 하고, 기말고사 얼마 안 남았으니까 샌쩍 같이 본 다음에 시간이 남으면 상반기 걸 같이 볼게요'},
  {'idx': 13, 'text': '그리고 제가 이거 답을 한번 봤는데 일단 풍거는 거의 다 맞았던 것 같아요'},
  {'idx': 14,
   'text': '네 그래서 풍거는 잘 푸는 것 같고 별표 위주로 한번 보면 될 것 같고 그 다음에 이게 지금 교재가 깨끗한 상태잖아요.'},
  {'idx': 15, 'text': '근데 하윤 학생이 뭘 틀렸는지 제가 모르니까.'},
  {'idx': 16, 'text': '이거를 하윤 학생이 그때그때 말해주면 좋을 것 같아요.'},
  {'idx': 17, 'text': '이 시간이 왜 지금 가고 있는거지?'},


In [200]:
from prompt import question_analyzer

# llm = ChatOpenAI(model='gpt-4o')
llm = ChatOpenAI(
    model='deepseek-chat', 
    openai_api_key=DEEPSEEK_API_KEY, 
    openai_api_base='https://api.deepseek.com',
)

importlib.reload(question_analyzer)
system_prompt = question_analyzer.QuestionAnalyzer(subject, user='선생님').prompt
prompt = ChatPromptTemplate.from_messages([
    ('system', system_prompt),
    ('user', "{user_message}")
])
chain = prompt | llm | StrOutputParser()

# Run it in parallel over chunks
question_analysis_results = chain.batch([
    {"user_message": chunk} for chunk in chunks_with_overlap
])

In [203]:
importlib.reload(utils)
question_indices = utils.extract_question_indices(question_analysis_results)
question_context = utils.get_question_context_v2(df, question_indices, 'teacher', 5)
question_context

[{'idx': 3,
  'question': '목소리 잘 들릴까요?',
  'context': [{'time': '0m 0.0s ~ 0m 2.4s', 'student_text': '감사합니다.'},
   {'time': '4m 41.8s ~ 5m 10.4s', 'student_text': '네 안녕하세요'},
   {'time': '4m 41.8s ~ 5m 10.4s', 'student_text': '네 네'},
   {'time': '4m 54.5s ~ 4m 58.3s', 'teacher_text': '목소리 잘 들릴까요?'},
   {'time': '4m 58.5s ~ 5m 0.6s', 'teacher_text': '안녕하세요.'},
   {'time': '5m 0.7s ~ 5m 6.2s', 'teacher_text': '잘 들리고 그러면 필기 한번 확인해 볼게요.'},
   {'time': '5m 9.7s ~ 5m 12.5s', 'teacher_text': '잘 보여요?'},
   {'time': '5m 12.5s ~ 5m 12.6s', 'teacher_text': '잠깐만요.'},
   {'time': '5m 23.3s ~ 5m 35.5s',
    'teacher_text': '혹시 전에 선택으로 과외 해본 적 있어요?'}]},
 {'idx': 6,
  'question': '잘 보여요?',
  'context': [{'time': '4m 41.8s ~ 5m 10.4s', 'student_text': '네 안녕하세요'},
   {'time': '4m 41.8s ~ 5m 10.4s', 'student_text': '네 네'},
   {'time': '4m 54.5s ~ 4m 58.3s', 'teacher_text': '목소리 잘 들릴까요?'},
   {'time': '4m 58.5s ~ 5m 0.6s', 'teacher_text': '안녕하세요.'},
   {'time': '5m 0.7s ~ 5m 6.2s', 'teacher_text': '잘 들리고 

In [204]:
from prompt import question_classifier

# llm = ChatOpenAI(model='gpt-4o')
llm = ChatOpenAI(
    model='deepseek-chat', 
    openai_api_key=DEEPSEEK_API_KEY, 
    openai_api_base='https://api.deepseek.com',
)

importlib.reload(question_classifier)
system_prompt = question_classifier.QuestionClassifier(subject, user='선생님').prompt
prompt = ChatPromptTemplate.from_messages([
    ('system', system_prompt),
    ('user', "{user_message}")
])
chain = prompt | llm | StrOutputParser()

question_classifier_results = chain.batch([
    {"user_message": chunk} for chunk in question_context
])

In [205]:
for (res,item) in zip(question_classifier_results, question_context):
    item['result'] = res
question_context

[{'idx': 3,
  'question': '목소리 잘 들릴까요?',
  'context': [{'time': '0m 0.0s ~ 0m 2.4s', 'student_text': '감사합니다.'},
   {'time': '4m 41.8s ~ 5m 10.4s', 'student_text': '네 안녕하세요'},
   {'time': '4m 41.8s ~ 5m 10.4s', 'student_text': '네 네'},
   {'time': '4m 54.5s ~ 4m 58.3s', 'teacher_text': '목소리 잘 들릴까요?'},
   {'time': '4m 58.5s ~ 5m 0.6s', 'teacher_text': '안녕하세요.'},
   {'time': '5m 0.7s ~ 5m 6.2s', 'teacher_text': '잘 들리고 그러면 필기 한번 확인해 볼게요.'},
   {'time': '5m 9.7s ~ 5m 12.5s', 'teacher_text': '잘 보여요?'},
   {'time': '5m 12.5s ~ 5m 12.6s', 'teacher_text': '잠깐만요.'},
   {'time': '5m 23.3s ~ 5m 35.5s', 'teacher_text': '혹시 전에 선택으로 과외 해본 적 있어요?'}],
  'result': 'False'},
 {'idx': 6,
  'question': '잘 보여요?',
  'context': [{'time': '4m 41.8s ~ 5m 10.4s', 'student_text': '네 안녕하세요'},
   {'time': '4m 41.8s ~ 5m 10.4s', 'student_text': '네 네'},
   {'time': '4m 54.5s ~ 4m 58.3s', 'teacher_text': '목소리 잘 들릴까요?'},
   {'time': '4m 58.5s ~ 5m 0.6s', 'teacher_text': '안녕하세요.'},
   {'time': '5m 0.7s ~ 5m 6.2s', 'teach

In [206]:
final_feed = [{'idx': item['idx'], 'question': item['question'], 'context': item['context']} for item in question_context if item['result'] == 'True']
final_feed

[{'idx': 32,
  'question': '근데 하윤 학생이 뭘 틀렸는지 제가 모르니까.',
  'context': [{'time': '6m 49.6s ~ 7m 13.3s',
    'teacher_text': '그리고 제가 이거 답을 한번 봤는데 일단 풍거는 거의 다 맞았던 것 같아요'},
   {'time': '409600 ~ 436300',
    'teacher_text': '네 그래서 풍거는 잘 푸는 것 같고 별표 위주로 한번 보면 될 것 같고 그 다음에 이게 지금 교재가 깨끗한 상태잖아요.'},
   {'time': '7m 1.4s ~ 7m 1.7s', 'student_text': '네.'},
   {'time': '7m 6.7s ~ 7m 6.7s', 'student_text': '네.'},
   {'time': '7m 9.7s ~ 7m 9.7s', 'student_text': '네.'},
   {'time': '7m 17.3s ~ 7m 20.2s',
    'teacher_text': '근데 하윤 학생이 뭘 틀렸는지 제가 모르니까.'},
   {'time': '7m 21.4s ~ 7m 27.6s',
    'teacher_text': '이거를 하윤 학생이 그때그때 말해주면 좋을 것 같아요.'},
   {'time': '7m 30.4s ~ 7m 34.2s', 'teacher_text': '이 시간이 왜 지금 가고 있는거지?'},
   {'time': '7m 40.5s ~ 7m 55.5s', 'student_text': '네.'},
   {'time': '7m 40.7s ~ 7m 43.0s', 'teacher_text': '지금 시작할게요.'},
   {'time': '7m 43.3s ~ 7m 45.6s', 'teacher_text': '자기소개부터 해볼게요.'}]},
 {'idx': 129,
  'question': '으 으 으 으 으 으 으 으 으 으 으 으 으 으 으 으 어떤 거부터 시작한다는 게 정확히 무슨 말이에요?',
  'conte

In [207]:
from prompt import digging_v2

llm = ChatOpenAI(
    model='deepseek-chat', 
    openai_api_key=DEEPSEEK_API_KEY, 
    openai_api_base='https://api.deepseek.com',
)

importlib.reload(digging_v2)
system_prompt = digging_v2.Digging(subject).prompt
prompt = ChatPromptTemplate.from_messages([
    ('system', system_prompt),
    ('user', "{user_message}")
])
chain = prompt | llm | StrOutputParser()

# Run it in parallel over chunks
results = chain.batch([
    {"user_message": chunk} for chunk in final_feed
])

In [208]:
results = [json.loads(item.strip('```json').strip('```').strip()) for item in results]

for (res, item) in zip(results, final_feed):
    item['result'] = res['result']
    item['reason'] = res['reason']
final_feed

[{'idx': 32,
  'question': '근데 하윤 학생이 뭘 틀렸는지 제가 모르니까.',
  'context': [{'time': '6m 49.6s ~ 7m 13.3s',
    'teacher_text': '그리고 제가 이거 답을 한번 봤는데 일단 풍거는 거의 다 맞았던 것 같아요'},
   {'time': '409600 ~ 436300',
    'teacher_text': '네 그래서 풍거는 잘 푸는 것 같고 별표 위주로 한번 보면 될 것 같고 그 다음에 이게 지금 교재가 깨끗한 상태잖아요.'},
   {'time': '7m 1.4s ~ 7m 1.7s', 'student_text': '네.'},
   {'time': '7m 6.7s ~ 7m 6.7s', 'student_text': '네.'},
   {'time': '7m 9.7s ~ 7m 9.7s', 'student_text': '네.'},
   {'time': '7m 17.3s ~ 7m 20.2s',
    'teacher_text': '근데 하윤 학생이 뭘 틀렸는지 제가 모르니까.'},
   {'time': '7m 21.4s ~ 7m 27.6s',
    'teacher_text': '이거를 하윤 학생이 그때그때 말해주면 좋을 것 같아요.'},
   {'time': '7m 30.4s ~ 7m 34.2s', 'teacher_text': '이 시간이 왜 지금 가고 있는거지?'},
   {'time': '7m 40.5s ~ 7m 55.5s', 'student_text': '네.'},
   {'time': '7m 40.7s ~ 7m 43.0s', 'teacher_text': '지금 시작할게요.'},
   {'time': '7m 43.3s ~ 7m 45.6s', 'teacher_text': '자기소개부터 해볼게요.'}],
  'result': '이해점검형 질문',
  'reason': "질문은 학생이 무엇을 틀렸는지 확인하기 위한 것으로, 학생이 자신의 오류를 설명하도록 요구하는 것이 아니라 단순히

In [273]:
with open(f"선생님_{room_id}.txt", "w", encoding="utf-8-sig") as file:
    json.dump(final_feed, file, ensure_ascii=False, indent=4)